# download libraries


In [1]:
# Requirements:
!pip install rtdl
!pip install libzero==0.0.4 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install openml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 119 kB 17.1 MB/s 
     |████████████████████████████████| 76 kB 6.0 MB/s 
  Created wheel for openml: filename=openml-0.12.2-py3-none-any.whl size=137326 sha256=a1eb2ead786bc32634726a10d68761a7e1a55714ec260d27eb601762432e9adb
  Stored in directory: /root/.cache/pip/wheels/6a/20/88/cf4ac86aa18e2cd647ed16ebe274a5dacee9d0075fa02af250
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11732 sha256=cfe2e3e178eead452c149537ffc8f8a8c53efb7c00e8aaa69eb126fe888dd428
  Stored in directory: /root/.cache/pip/wheels/1f/0f/15/332ca86cbebf25ddf98518caaf887945fbe1712b97a0f2493b
Successfully built openml liac-arff


In [3]:
!pip install wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 31.7 MB/s 
     |████████████████████████████████| 181 kB 66.3 MB/s 
     |████████████████████████████████| 157 kB 64.0 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 157 kB 71.1 MB/s 
     |████████████████████████████████| 157 kB 77.7 MB/s 
     |████████████████████████████████| 157 kB 72.5 MB/s 
     |████████████████████████████████| 157 kB 51.8 MB/s 
     |████████████████████████████████| 156 kB 72.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=f125d489e5d44c4746c5d9be0bedcf8332ea731542abe1034b9c004669de0a8e
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [4]:
# Requirements:
!pip install rtdl
!pip install libzero==0.0.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 29.8 MB/s 


#Auxiliary classes 

In [6]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable

import numpy as np
import openml
import pandas as pd
from openml import OpenMLDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

task_ids = (
    40588,
    40589,
    40590,
    40591,
    40592,
    40593,
    40594,
    40595,
    40596,
    40597,
)


@dataclass
class Split:
    X: np.ndarray
    y: np.ndarray


@dataclass
class Dataset:
    name: str
    id: int
    features: pd.DataFrame
    labels: pd.DataFrame
    openml: OpenMLDataset
    encoders: dict[str, LabelEncoder]

    def split(
        self,
        splits: Iterable[float],
        seed: int | None = 1,
    ) -> tuple[Split, ...]:
        """Create splits of the data

        Parameters
        ----------
        splits : Iterable[float]
            The percentages of splits to generate

        seed : int | None = None
            The seed to use for the splits

        Returns
        -------
        tuple[Split, ...]
            The collected splits
        """
        splits = list(splits)
        assert abs(1 - sum(splits)) <= 1e-6, "Splits must sum to 1"

        sample_sizes = tuple(int(s * len(self.features)) for s in splits)

        collected_splits = []

        next_xs = self.features.to_numpy()
        next_ys = self.labels.to_numpy()

        for size in sample_sizes[:-1]:
            xs, next_xs, ys, next_ys = train_test_split(
                next_xs, next_ys, train_size=size, random_state=seed
            )
            collected_splits.append(Split(X=xs, y=ys))
        collected_splits.append(Split(X=next_xs, y=next_ys))

        return tuple(collected_splits)

    @staticmethod
    def from_openml(id: int) -> Dataset:
        """Processes an multilabel OpenMLDataset into its features and targets

        Parameters
        ----------
        id: int
            The id of the dataset

        Returns
        -------
        Dataset
        """
        dataset = openml.datasets.get_dataset(id)
        print(dataset.name, id)
        targets = dataset.default_target_attribute.split(",")
        data, _, _, _ = dataset.get_data()

        assert isinstance(data, pd.DataFrame)

        # Process the features and turn all categorical columns into ints
        features = data.drop(columns=targets)
        encoders: dict[str, LabelEncoder] = {}

        for name, col in features.iteritems():
            if col.dtype in ["object", "category", "string"]:
                encoder = LabelEncoder()
                features[name] = encoder.fit_transform(col)
                encoders[name] = encoder

        labels = data[targets]

        # Since we assume binary multilabel data, we convert the labels
        # to all be boolean types
        labels = labels.astype(bool)

        return Dataset(
            name=dataset.name,
            id=id,
            features=features,
            labels=labels,
            openml=dataset,
            encoders=encoders,
        )

from sklearn.metrics import f1_score as sklearn_f1_score
import numpy as np

def f1_score(y_true: np.ndarray, y_pred:np.ndarray) -> float:
    return sklearn_f1_score(y_true, y_pred, average="macro", zero_division=0)

In [7]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

# get the dataset
def get_dataset():
	X, y = make_multilabel_classification(n_samples=1000, n_features=10, n_classes=3, n_labels=2, random_state=1)
	return X, y

# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(20, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# make a prediction on the test set
		yhat = model.predict(X_test)
		# round probabilities to class labels
		yhat = yhat.round()
		# calculate accuracy
		acc = f1_score(y_test, yhat)
		# store result
		print('>%.3f' % acc)
		results.append(acc)
	return results



In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [9]:
from torch.functional import split
class MyDataset(Dataset):
 
  def __init__(self,split):
 
    x=np.array(split.X)
    y=np.array(split.y)
 
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y_train=torch.tensor(y,dtype=torch.float32)
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [10]:
def train_(dataloader, model, loss_fn, optimizer,logger,dataset_test=None,ftt=False):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X,None) if ftt else model(X) 
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
          yhat = torch.sigmoid(pred).round()
          f1_train = f1_score(y.detach().cpu().numpy(), yhat.detach().cpu().numpy())
          if(dataset_test is not None):
            f1_test = test(dataset_test,model,loss_fn,ftt)
            logger.log({"loss":loss,"F1_train":f1_train,"F1_val":f1_test})
          else:
            logger.log({"loss":loss,"F1_train":f1_train})
        del X,y,pred,loss

def train(dataloader, model, loss_fn, optimizer,dataset_test=None,ftt=False):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X,None) if ftt else model(X) 
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       

def test(dataloader, model, loss_fn,ftt=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    #print("num batches: "+str(num_batches))
    #print("size: "+str(size))
    model.eval()
    f1 = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X,None) if ftt else  model(X ) 
            pred = torch.sigmoid(pred)
            yhat = pred.round()
            # calculate accuracy
            f1 += f1_score(y.detach().cpu().numpy(), yhat.detach().cpu().numpy())
    f1 /= num_batches
    return f1
    

In [11]:
def get_best_thresholds(dataloader,model,ftt=False):
  model.eval()
  with torch.no_grad():
      for X, y in dataloader:
          X, y = X.to(device), y.to(device)
          pred = model(X,None) if ftt else  model(X)
          pred = torch.sigmoid(pred)
  final_thresholds = np.array([])
  thresholds = np.array([.20,.40,.50,.60,80])
  for i in range(y.shape[1]):
    pred_ =pred[:,i]
    y_ = y[:,i]
    scores = np.array([])
    for threshold in thresholds:
      yhat = (pred_>threshold).float()
      score = f1_score(yhat.detach().cpu().numpy(),y_.detach().cpu().numpy())
      scores = np.append(scores,score)
    index = np.argmax(scores)
    final_thresholds = np.append(final_thresholds,thresholds[index])
  return final_thresholds

In [12]:
def test_with_thresholds(dataloader, model, loss_fn,thresholds,ftt=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    f1 = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X,None) if ftt else  model(X) 
            pred = torch.sigmoid(pred)
            yhat = None
            for i in range(len(thresholds)):
              pred_ =pred[:,i]
              yhat_ = (pred_>thresholds[i]).float()
              if yhat== None:
                yhat = yhat_ 
              else: 
                yhat = torch.column_stack((yhat,yhat_))
            f1 = f1_score(y.detach().cpu().numpy(), yhat.detach().cpu().numpy())
    f1 /= num_batches
    return f1


In [13]:
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

total    : 15843721216
free     : 15840575488
used     : 3145728


In [14]:
import rtdl
x = torch.randn(4, 2)
config = { "d_in":x.shape[1],
    "n_blocks":5,
    "d_main":512,
    "d_hidden":512,
    "dropout_first":0.35,
    "dropout_second":0.0,
    "d_out":1}
module = rtdl.ResNet.make_baseline(**config)
assert module(x).shape == (len(x), 1)

print(module)

ResNet(
  (first_layer): Linear(in_features=2, out_features=512, bias=True)
  (blocks): Sequential(
    (0): Block(
      (normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (linear_first): Linear(in_features=512, out_features=512, bias=True)
      (activation): ReLU()
      (dropout_first): Dropout(p=0.35, inplace=False)
      (linear_second): Linear(in_features=512, out_features=512, bias=True)
      (dropout_second): Dropout(p=0.0, inplace=False)
    )
    (1): Block(
      (normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (linear_first): Linear(in_features=512, out_features=512, bias=True)
      (activation): ReLU()
      (dropout_first): Dropout(p=0.35, inplace=False)
      (linear_second): Linear(in_features=512, out_features=512, bias=True)
      (dropout_second): Dropout(p=0.0, inplace=False)
    )
    (2): Block(
      (normalization): BatchNorm1d(512, eps=1e-05, momentum=

#Data Preprocesing 

In [15]:
import os
import numpy as np
import sys
import time
import re
import math

np.set_printoptions(threshold=sys.maxsize)

class Dataset_():
    def __init__(self, dataset_number, threshold=0.7, seed = 0) -> None:
        self.features_name = []
        self.features_type = []
        self.features_possible_values = []
        self.features_unique_value = []
        self.data = []
        self.random_generator = np.random.RandomState(seed)
        self.threshold = threshold
        self.dataset_number_to_name(dataset_number)
        
        self.initialization()
    
    def load_dataset(self):
        
        # Get path
        absolute_path = os.getcwd()
        full_path = os.path.join(absolute_path, "gdrive/MyDrive/datasets", self.dataset_name+".arff")
        
        # Read file
        with open(full_path, 'r') as f:
            lines = f.readlines()
        
        # Load dataset by reading file line by line
        state = 0
        for line in lines:
            strings = line[:-1].split(" ", 2)
            if strings[0] == "@relation":
                print("Begin to read dataset")
                
            elif strings[0] == "@attribute":
                self.features_name.append(strings[1][1:-1])
                if strings[2][0] == "{" and strings[2][-1] == "}":
                    if len(strings[2].split(",")) == 2:
                        self.features_type.append("binary")
                    else:
                        self.features_type.append("categorical")
                else:
                    self.features_type.append(strings[2])
                
                if self.features_type[-1] == "categorical" or self.features_type[-1] == "binary":
                    values = strings[2][1:-1].replace(" ", "")
                    values = values.replace("'", "")
                    values = values.split(",")
                    self.features_possible_values.append(values)
                else :
                    self.features_possible_values.append("endless")
                
            elif strings[0] == "@data":
                # From now Read data
                state = 1
            
            elif state == 1:
                tmp_line = line[:-1].replace('"', "")
                tmp_line = tmp_line.replace(' ', "")
                values = tmp_line.split(",")
                sample = []
                for i in range(len(values)):
                    value = values[i]
                    if self.features_type[i] == "numeric":
                        sample.append(float(value))
                    
                    elif self.features_type[i] == "binary" or self.features_type[i] == "categorical":
                        value = self.features_possible_values[i].index(value)
                        sample.append(float(value))
                self.data.append(sample)
            else:
                assert False, "Got to invalid case: state "+str(state)+" first string "+strings[0]                                
                
                
        self.data = np.array(self.data)
        for i in np.arange(self.data.shape[1]):
            self.features_unique_value.append(np.unique(self.data[:,i]))
            if len(self.features_unique_value[i]) == 2 and self.features_type == "numeric":
                self.features_type[i] = "binary"
                self.features_possible_values = self.features_unique_value
                for j in range(self.data.shape[0]):
                    self.data[j,i] = float(self.features_possible_values[i].index(value))
                

        
    def check_for_single_value_parameters(self):
        
        params_to_delete = []
        for i in np.arange(self.data.shape[1]-1, -1, -1):
            if self.features_name[i] in self.target_features:
                continue
            unique_values = self.features_unique_value[i]
            if unique_values.shape[0] == 1:
                params_to_delete.append(i)

        old = self.data.shape[1]
        self.data = np.delete(self.data, params_to_delete, 1)
        self.clear_feature_data(params_to_delete)
        new = self.data.shape[1]
        print("Removed ",old - new," parameters with one unique value")
        #print(params_to_delete)
    
    def check_for_pseudocorrelation(self):
        
        params_to_delete = []
        for i in np.arange(self.data.shape[1]-1, -1, -1):
            if self.features_name[i] in self.target_features or self.features_type[i] == "numeric":
                continue
            for j in np.arange(i-1, -1, -1):
                if self.features_name[j] in self.target_features or self.features_type[j] == "numeric":
                    continue
                # Check for pseudocorrelation
                temp_i = self.data[:,i]
                temp_j = self.data[:,j]
                uniq_val_num_i = len(self.features_unique_value[i])
                uniq_val_num_j = len(self.features_unique_value[j])
                if uniq_val_num_i > uniq_val_num_j:
                    lower = j
                else:
                    lower = i
                temp_i = temp_i * uniq_val_num_j
                sum = temp_i + temp_j
                uniq_val = np.unique(sum)
                if uniq_val.shape[0] == uniq_val_num_i or uniq_val.shape[0] == uniq_val_num_j:
                    if not(lower in params_to_delete):
                        params_to_delete.append(lower)
                        #print(self.features_name[i]," ",self.features_name[j]," ",lower)
                        
                    #break

        old = self.data.shape[1]
        self.data = np.delete(self.data, params_to_delete, 1)
        self.clear_feature_data(params_to_delete)
        new = self.data.shape[1]
        print("Removed ",old - new," parameters with pseudocorrelated values")
        #print(params_to_delete)
              
              
    def check_for_correlation(self):
        params_to_delete = []
        for i in range(len(self.features_type)):
            if self.features_name[i] in self.target_features or self.features_type[i] != "numeric":
                continue
            for j in range(len(self.features_type)):
                if self.features_name[j] in self.target_features or self.features_type[j] == "numeric":
                    continue
                
                sum_i = np.sum(self.data[:, i])
                sum_j = np.sum(self.data[:, j])
                sum_i2 = np.sum(self.data[:, i]**2)
                sum_j2 = np.sum(self.data[:, j]**2)
                sum_ij = np.sum(self.data[:, i] * self.data[:, j])
                n = self.data.shape[0]
                
                correlation = (n*sum_ij - sum_i * sum_j)/(math.sqrt((n * sum_i2 - (sum_i)**2)*(n * sum_j2 - (sum_j)**2)))
                correlation = abs(correlation)
                if correlation>= self.threshold:
                    if len(self.features_unique_value[i])>=len(self.features_unique_value[j]):
                        if not(j in params_to_delete):
                            params_to_delete.append(j)
                    else:
                        if not(i in params_to_delete):
                            params_to_delete.append(i)
                
        old = self.data.shape[1]
        self.data = np.delete(self.data, params_to_delete, 1)
        self.clear_feature_data(params_to_delete)
        new = self.data.shape[1]
        print("Removed ",old - new," parameters with correlated values")
        #print(params_to_delete)
                 
    def clear_feature_data(self, indices:list):
        
        indices.sort()
        indices = indices[::-1]
        for i in indices:
            self.features_name.pop(i)
            self.features_type.pop(i)
            self.features_possible_values.pop(i)
            self.features_unique_value.pop(i)   
        
        
    def find_targets(self):
        
        if self.dataset_name == "birds":
            pattern = r"^[A-Z][a-z]"
            number_of_targets = 19
        elif self.dataset_name == "emotions":
            pattern = r"[a-z]\.[a-z]"
            number_of_targets = 6
        elif self.dataset_name == "enron":
            pattern = r"[A-Z]\.[A-Z][0-9]+"
            number_of_targets = 53
        elif self.dataset_name == "genbase":
            pattern = r"PDOC"
            number_of_targets = 27
        elif self.dataset_name == "image":
            pattern = r"^((?!Feature).)*$"
            number_of_targets = 5
        elif self.dataset_name == "langLog":
            pattern = r"^((?!tok[0-9]+).)*$"
            number_of_targets = 75
        elif self.dataset_name == "reuters":
            pattern = r"^label[1-7]"
            number_of_targets = 7
        elif self.dataset_name == "scene":
            pattern = "^((?!Att[0-9]).)*$"
            number_of_targets = 6
        elif self.dataset_name == "slashdot":
            pattern = "^[A-Z][a-zA-Z]"
            number_of_targets = 22
        elif self.dataset_name == "yeast":
            pattern = r"Class[1-9]"
            number_of_targets = 14
        else: 
            assert False, "Wrong dataset name : "+self.dataset_name
        
        self.target_features = []
        for i in range(self.data.shape[1]-1, -1, -1):
            decision = re.search(pattern, self.features_name[i])
            if decision:
                self.target_features.append(self.features_name[i])
                self.features_name
        self.number_of_targets = number_of_targets
        assert len(self.target_features) == self.number_of_targets, "Found number of targets "+str(len(self.target_features))+" differs from true number "+str(self.number_of_targets) 
        
        
    def check_for_values_with_one_occurance(self):
        
        params_to_delete = []
        for i in np.arange(self.data.shape[1]-1, -1, -1):
            if self.features_name[i] in self.target_features:
                continue
            if self.features_type[i] == "numeric":
                continue
            temp = (self.data[:,i]).astype(int)
            occurances = np.bincount(temp)
            if 1 in occurances:
                params_to_delete.append(i)

        old = self.data.shape[1]
        self.data = np.delete(self.data, params_to_delete, 1)
        self.clear_feature_data(params_to_delete)
        new = self.data.shape[1]
        print("Removed ",old - new," parameters with 1 occurance of value")
        #print(params_to_delete)
    
    
    def check_for_duplicated_samples(self):
        
        old = self.data.shape[0]
        self.data = np.unique(self.data, axis=0)
        new = self.data.shape[0]
        print("Removed ",old - new," samples due to being duplicates")
    
    
    def normalize_dataset(self):
        
        for i in range(self.data.shape[1]):
            if self.features_name in self.target_features:
                continue
            if not self.features_type == "numeric":
                continue
            
            average = np.mean(self.data[:, i])
            
            self.data[:, i] = self.data[:, i] - average
            
            maximum = np.amax(np.absolute(self.data[:,i]))
            
            self.data[:, i] = self.data[:, i]/maximum
            
    
    def dataset_split(self, train_ratio : float):
        
        perm_idxs = np.arange(self.data.shape[0])
        self.random_generator.shuffle(perm_idxs)
        split_idx = int(self.data.shape[0]*train_ratio)
        train_idxs = perm_idxs[:split_idx]
        valid_idxs = perm_idxs[split_idx:]
        
        train_pred_data = self.predictors[train_idxs]
        train_tar_data = self.targets[train_idxs]
        valid_pred_data = self.predictors[valid_idxs]
        valid_tar_data = self.targets[valid_idxs]
        return train_pred_data, train_tar_data, valid_pred_data, valid_tar_data
        
       
    def dataset_to_predictors_targets(self):
        
        self.predictors = []
        self.targets = []
        self.predictors_features_type = []
        self.targets_features_type = []
        
        for i in range(self.data.shape[1]):
            if self.features_name[i] in self.target_features:
                self.targets.append(self.data[:,i])
                self.targets_features_type.append(self.features_type[i])
            else:
                self.predictors.append(self.data[:,i])
                self.predictors_features_type.append(self.features_type[i])
                
        self.predictors = np.array(self.predictors)
        self.targets = np.array(self.targets)
        self.targets = np.swapaxes(self.targets, 0, 1)
        self.predictors = np.swapaxes(self.predictors, 0, 1)
        
    
    def dataset_number_to_name(self, number):
        dataset_names = ["birds", "emotions", "enron", "genbase",
                         "image", "langLog", "reuters", "scene",
                         "slashdot", "yeast"]
        
        assert number < len(dataset_names), "Chosen number "+str(number)+" is outside possible values 0-"+str(len(dataset_names)-1)
        self.dataset_name = dataset_names[number]
        print(dataset_names[number])
            
        
    def one_hot_encoding(self, x):
        length = x.shape[0]
        max_value = int(np.max(x))
        new_array = np.zeros((length, max_value+1))
        new_array[np.arange(length), x.astype(int)] = 1.0
        return new_array
    
    
    def dataset_to_one_hot_encoding(self):
        for i in range(len(self.predictors_features_type)-1, -1, -1):
            if self.predictors_features_type[i] == "categorical":
                ar = self.one_hot_encoding(self.predictors[:,i])
            else:
                ar = self.predictors[:,i]
                ar = np.reshape(ar, (-1, 1))

            if i+1 == len(self.predictors_features_type):
                self.predictors = np.concatenate((self.predictors[:,:i], ar),axis=1)
            else:
                self.predictors = np.concatenate((self.predictors[:,:i], ar, self.predictors[:,i+1:]),axis=1)
        
        for i in range(len(self.targets_features_type)-1, -1, -1):
            if self.targets_features_type[i] == "categorical":
                ar = self.one_hot_encoding(self.targets[:,i])
            else:
                ar = self.targets[:,i]
                ar = np.reshape(ar, (-1, 1))
            if i+1 == len(self.targets_features_type):
                self.targets = np.concatenate((self.targets[:,:i], ar),axis=1)
            else:
                self.targets = np.concatenate((self.targets[:,:i], ar, self.targets[:,i+1:]),axis=1)
            
    def initialization(self):
        start = time.process_time()
        self.load_dataset()
        self.find_targets()
        self.check_for_single_value_parameters()
        self.check_for_values_with_one_occurance()
        self.check_for_duplicated_samples()
        self.check_for_pseudocorrelation()
        self.check_for_correlation()
        self.normalize_dataset()
        self.dataset_to_predictors_targets()
        self.dataset_to_one_hot_encoding()
        
        print("It took ",time.process_time() - start)
        print("Final shape of the dataset (predictors + tagrets) is ",self.data.shape)
        print("Number of predictors : ", self.data.shape[1] - len(self.target_features), " and number of targets : ",len(self.target_features))





class DatasetSampler():
    def __init__(self, train_pred, train_target, valid_pred, valid_target, train_bs = 64, valid_bs = 64, seed = 0, reshuffle = False, mode = "train") -> None:
        self.train_pred = train_pred
        self.train_target = train_target
        self.train_bs = train_bs
        
        self.valid_pred = valid_pred
        self.valid_target = valid_target
        self.valid_bs = valid_bs
        
        self.random_generator = np.random.RandomState(seed)
        self.reshuffle = reshuffle
        
        self.train_iter = 0
        self.valid_iter = 0
        
        self.train_iter_max = math.ceil(self.train_pred.shape[0] / self.train_bs)
        self.valid_iter_max = math.ceil(self.valid_pred.shape[0] / self.valid_bs)
        
        assert mode == "train" or mode == "valid", "Chosen mode is not train or valid. Chosen mode is"+mode
        self.mode = mode
        
        
    def sample_batch(self, pred, target, bs, iter):
             
        pred_batch = pred[iter*bs : min((iter+1)*bs ,pred.shape[0])]
        target_batch = target[iter*bs : min((iter+1)*bs ,pred.shape[0])]
        return pred_batch, target_batch 
    
    
    def get_batch(self):
                
        if self.mode == "train":
            if self.train_iter == self.train_iter_max :
                if self.reshuffle:
                    perm = np.arange(self.train_pred.shape[0])
                    self.random_generator.shuffle(perm)
                    self.train_pred = self.train_pred[perm]
                    self.train_target = self.train_target[perm]
                self.train_iter = 0
                raise StopIteration
                
            pred_batch, target_batch = self.sample_batch(self.train_pred, self.train_target, self.train_bs, self.train_iter)  
            self.train_iter = self.train_iter + 1
            
        elif self.mode == "valid":
            if self.valid_iter == self.valid_iter_max :
                if self.reshuffle:
                    perm = np.arange(self.valid_pred.shape[0])
                    self.random_generator.shuffle(perm)
                    self.valid_pred = self.valid_pred[perm]
                    self.valid_target = self.valid_target[perm]
                self.valid_iter = 0
                raise StopIteration
            
            pred_batch, target_batch = self.sample_batch(self.valid_pred, self.valid_target, self.valid_bs, self.valid_iter)
            self.valid_iter = self.valid_iter + 1
            
        return pred_batch, target_batch
        
    
    def reset_iterators(self):
        self.train_iter = 0
        self.valid_iter = 0
        
    
    def set_mode(self, mode):
        assert mode == "train" or mode == "valid", "Chosen mode is not train or valid. Chosen mode is"+mode
        self.mode = mode
        
    def __iter__(self):
        return self
    
    
    def __next__(self):
        return self.get_batch()
    
    
    def __getitem__(self, index):
        
        assert self.mode != "train" or self.train_iter_max >= index, "Index outside range :"+str(self.train_iter_max)+" got "+str(index)
        assert self.mode != "valid" or self.valid_iter_max >= index, "Index outside range :"+str(self.valid_iter_max)+" got "+str(index)
        
        if self.mode == "train":
            temp = self.train_iter
            to_return = self.get_batch()
            self.train_iter = temp
            
        elif self.mode == "valid":
            temp = self.valid_iter
            to_return = self.get_batch()
            self.valid_iter = temp
            
        else:
            assert False, "Invalid mode :"+self.mode
            
        return to_return
            
    def __len__(self):
        if self.mode == "train":
            to_return = self.train_iter_max
            
        elif self.mode == "valid":
            to_return = self.valid_iter_max
            
        else:
            assert False, "Invalid mode :"+self.mode
    
        return to_return


#DE

In [16]:
import matplotlib.pyplot as plt
import rtdl
import wandb


In [17]:

def obj(x,budget,dataset_id):

  seed = 42

  loss_fn = nn.BCEWithLogitsLoss()

  dataset = Dataset.from_openml(dataset_id)
  train_split, test_split = dataset.split(splits=(0.75, 0.25), seed=seed)
  n_inputs, n_outputs = len(dataset.features.columns), len(dataset.labels.columns)

  name = "ResNet_DEHB_" + dataset.name
  config = { "d_in":n_inputs,
    "n_blocks":x["n_blocks"],
    "d_main":x["d_main"],
    "d_hidden":x["d_hidden"],
    "dropout_first":x["dropout_first"],
    "dropout_second":x["dropout_second"],
    "d_out":n_outputs}


  model = rtdl.ResNet.make_baseline(**config).to(device)  

  config["batch_size"] = x["batch_size"]
  config["epoch"] = budget

  print(config)

  optimizer = torch.optim.AdamW(model.parameters(), lr=x['lr'], weight_decay=x['weight_decay'])

  train_dataset = MyDataset(train_split)
  test_dataset = MyDataset(test_split)

  train_dataloader = DataLoader(train_dataset, batch_size=x["batch_size"])
  test_dataloader = DataLoader(test_dataset, batch_size=x["batch_size"])

  for t in range(budget):
    train(train_dataloader, model, loss_fn, optimizer,ftt=False,dataset_test=test_dataloader)
  
  score_test = test(test_dataloader,model, loss_fn,ftt=False)

  del model
  torch.cuda.empty_cache()

  print("f1_score: " + str(score_test))
  
  return {"f1_score": score_test}

In [ ]:
import json
import os
import time
from typing import Callable, Union

import ConfigSpace
import numpy as np
from ConfigSpace.util import (deactivate_inactive_hyperparameters,
                              impute_inactive_values)

SEED = 42


class ConfigVectorSpace(ConfigSpace.ConfigurationSpace):

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.hyperparameters = self.get_hyperparameters()
        self.dim = len(self.hyperparameters)

        self.name_to_idx = {}
        for i, hp in enumerate(self.hyperparameters):
            # maps hyperparameter name to positional index in vector form
            self.name_to_idx[hp.name] = i
        
    def sample_vectors(self, size):
        configurations = super().sample_configuration(size)
        if not isinstance(configurations , list):
            configurations = [configurations]

        vectors = [self._to_vector(config) for config in configurations]
        return vectors
    
    def _to_vector(self, config: ConfigSpace.Configuration) -> np.array:
        '''Converts ConfigSpace.Configuration object to numpy array scaled to [0,1]
        Works when self is a ConfigVectorSpace object and the input config is a ConfigSpace.Configuration object.
        Handles conditional spaces implicitly by replacing illegal parameters with default values
        to maintain the dimensionality of the vector.
        '''        
        config = impute_inactive_values(config)

        vector = [None] * self.dim
        self.name_to_id = dict()

        for name in config:
            idx = self.name_to_idx[name]
            hyper = self[name]
            if type(hyper) == ConfigSpace.OrdinalHyperparameter:
                nlevels = len(hyper.sequence)
                vector[idx] = hyper.sequence.index(config[name]) / nlevels
            elif type(hyper) == ConfigSpace.CategoricalHyperparameter:
                nlevels = len(hyper.choices)
                vector[idx] = hyper.choices.index(config[name]) / nlevels
            else:
                bounds = (hyper.lower, hyper.upper)
                param_value = config[name]
                if hyper.log:
                    vector[idx] = np.log(param_value / bounds[0]) / np.log(bounds[1] / bounds[0])
                else:
                    vector[idx] = (config[name] - bounds[0]) / (bounds[1] - bounds[0])
        return np.array(vector)
    
    def to_config(self, vector: np.array) -> ConfigSpace.Configuration:
        '''Converts numpy array to ConfigSpace.Configuration object
        Works when self is a ConfigVectorSpace object and the input vector is in the domain [0, 1].
        '''
        # creates a ConfigSpace object dict with all hyperparameters present, the inactive too
        new_config = impute_inactive_values(self.sample_configuration()).get_dictionary()

        # iterates over all hyperparameters and normalizes each based on its type
        for hyper in self.hyperparameters:
            idx = self.name_to_idx[hyper.name]

            if type(hyper) == ConfigSpace.OrdinalHyperparameter:
                ranges = np.arange(start=0, stop=1, step=1/len(hyper.sequence))
                param_value = hyper.sequence[np.where((vector[idx] < ranges) == False)[0][-1]]
            elif type(hyper) == ConfigSpace.CategoricalHyperparameter:
                ranges = np.arange(start=0, stop=1, step=1/len(hyper.choices))
                param_value = hyper.choices[np.where((vector[idx] < ranges) == False)[0][-1]]
            else:  # handles UniformFloatHyperparameter & UniformIntegerHyperparameter
                # rescaling continuous values
                if hyper.log:
                    log_range = np.log(hyper.upper) - np.log(hyper.lower)
                    param_value = np.exp(np.log(hyper.lower) + vector[idx] * log_range)
                else:
                    param_value = hyper.lower + (hyper.upper - hyper.lower) * vector[idx]
                if type(hyper) == ConfigSpace.UniformIntegerHyperparameter:
                    param_value = int(np.round(param_value))  # converting to discrete (int)
                else:
                    param_value = float(param_value)
            new_config[hyper.name] = param_value

        #self.check_configuration(ConfigSpace.Configuration(self, values=new_config))
        #new_config = impute_inactive_values(ConfigSpace.Configuration(self, values=new_config))
        # the mapping from unit hypercube to the actual config space may lead to illegal
        # configurations based on conditions defined, which need to be deactivated/removed
        new_config = deactivate_inactive_hyperparameters(
            configuration = new_config, configuration_space=self
        )
        return new_config


class DE(object):
    def __init__(self,
        space : ConfigVectorSpace,
        crossover_prob : float = 0.9,
        mutation_factor : float = 0.8,
        metric = "f1_score",
        mode = "max",
        rs: np.random.RandomState=None,
        bound_control = "random",
        save_path=".") -> None:

        assert 0 <= mutation_factor <= 2, ValueError("mutation_factor not in range [0, 2]")
        assert mode in ["min", "max"], ValueError("Valid optimization mode in ['min', 'max']")
        
        self.space = space
        self.crossover_prob = crossover_prob
        self.mutation_factor = mutation_factor
        self.metric = metric
        self.mode = mode
        self.rs = rs
        self.bound_control = bound_control
        self.save_path=save_path
        
        self.traj = []
        self.inc_config = None
        self.inc_score = float("inf") if self.mode == "min" else float("-inf")

        self.histroy = []

        self._min_pop_size = 3
        self._eval_counter = -1
        self._iteration_counter = -1

    def _init_population(self, pop_size : int) -> np.ndarray :

        # sample from ConfigSpace s.t. conditional constraints (if any) are maintained
        population = self.space.sample_vectors(size=pop_size)
        return np.asarray(population)
    
    def _eval_population(self, obj : Callable, population: Union[np.ndarray, list] , budget, **kwargs) -> np.ndarray:
        fitness = []

        for candidate in population:
            config = self.space.to_config(candidate)
            result = obj(config, budget, **kwargs)

            assert isinstance(result, dict), TypeError("Objective function must return a dictionary")
            assert self.metric in result, KeyError(f"Given mteric '{self.metric}' not found in dictionary {result}, returned by the objective function")
            score = result[self.metric]

            condition = {
                "min" : score < self.inc_score,
                "max" : score > self.inc_score, 
            } 

            if condition[self.mode]:
                self.inc_config = config
                self.inc_score = score

            fitness.append(score)
            
            self.traj.append(self.inc_score)
            self._update_history(candidate, result, budget)
            self._eval_counter += 1

        return np.asarray(fitness)

    def _update_history(self, candidate, result, budget):
        record = {
            "candidate": candidate.tolist(),
            "budget": budget}
        
        record.update(result)
        self.histroy.append(record)

    def _sample(self, population, size, replace=False):
        selection = self.rs.choice(np.arange(len(population)), size, replace=replace)
        return population[selection]

    def _mutation(self, population: np.ndarray):
        pop_size = len(population)
        assert pop_size > self._min_pop_size, ValueError(f"Population too small ( < DE()._min_pop_size = {self._min_pop_size}) for mutation")

        base, a, b = self._sample(population, self._min_pop_size)

        diff = a - b
        mutant = base + self.mutation_factor * diff
        return mutant

    def _crossover(self, candidate: np.ndarray, mutant: np.ndarray) -> np.ndarray:
        # Sample a random value from U[0, 1] for every dimension
        p = self.rs.rand(self.space.dim)

        # perform binomial crossover
        child = np.asarray([mutant[i] if p[i] < self.crossover_prob else candidate[i] for i in range(self.space.dim)])
        return child
    
    def _selection(self, population: np.ndarray, children: np.ndarray, fitness: np.ndarray, children_fitness: np.ndarray):
        # Conduct parent-child competition and select new population 
        pop_size = len(population)
        for i in range(pop_size):
            condition = {
                "max" : children_fitness[i] >= fitness[i],
                "min" : children_fitness[i] <= fitness[i]}
            if condition[self.mode]:
                population[i] = children[i]
                fitness[i] = children_fitness[i]
        
        return population, fitness
    
    def _check_bounds(self, vector: np.ndarray) -> np.ndarray:

        violations = np.where((vector > 1) | (vector < 0))[0]
        if len(violations) == 0:
            return vector
        if self.bound_control == 'random':
            vector[violations] = self.rs.uniform(low=0.0, high=1.0, size=len(violations))
        else:
            # Can be exploited by optimizer if solution at clip limits 
            vector[violations] = np.clip(vector[violations], a_min=0, a_max=1)
        return vector

    def _next_generation(self, population, alt_pop=None):
        children = []

        for candidate in population:
            # If alt_pop is None, vanilla mutation is perfomed
            mutant = self._mutation(alt_pop if alt_pop is not None else population)
            mutant = self._check_bounds(mutant)
            child = self._crossover(candidate, mutant)
            children.append(child)
        
        return children

    def optimize(self, obj : Callable, budget=None, pop_size : Union[int,None] = None, limit: int = 10, unit : str = "iter", **kwargs):
        self._start_timer()

        if pop_size is None:
            pop_size = 10 * self.space.dim # heuristic

        # Initialize and Evaluate the population
        population = self._init_population(pop_size)
        fitness = self._eval_population(obj, population, budget, **kwargs)

        # Until budget is exhausted
        while not self._is_termination(limit, unit):
            self._iteration_counter += 1

            children = self._next_generation(population)

            children_fitness = self._eval_population(obj, children, budget)
            population, fitness = self._selection(population, children, fitness, children_fitness)
        
        return self.inc_config
    
    def _start_timer(self):
        self._wall_clock_start = time.time()

    def _is_termination(self, limit : int, unit : str):
        assert unit in ["hr", "min", "sec", "evals", "iter"], ValueError("Unrecognized unit given")

        if unit in ["hr", "min", "sec"]:
            scale = {
                "sec" : 1,
                "min" : 60,
                "hr"  : 60 * 60
            }
            diff = time.time() - self._wall_clock_start
            return diff >= limit * scale[unit]
        elif unit == "evals":
            return self._eval_counter + 1 > limit
        else:
            return self._iteration_counter + 1 > limit

    def save_data(self):
        data = {
            "params" : self._init_params(),
            "result" : {
                "best_config": self.inc_config.get_dictionary(),
                "best_score" : self.inc_score,
            },
            "traj": self.traj,
            "history" : self.histroy,
        }

        with open(os.path.join(self.save_path, "data.json"), "w") as outfile:
            json.dump(data, outfile)
    
    def _init_params(self):
        params = {
            "crossover_prob" : self.crossover_prob,
            "mutation_factor" : self.mutation_factor,
            "metric" : self.metric,
            "mode" : self.mode,
            "seed" : SEED,
            "bound_control" : self.bound_control,
            "iters" : self._iteration_counter,
            "evals" : self._eval_counter

        }
        return params

class DEHB(DE):
    def __init__(self, 
        space: ConfigVectorSpace,
        min_budget : int = 100,
        max_budget : int = 1000,
        eta : int = 2,
        crossover_prob: float = 0.9, 
        mutation_factor: float = 0.8, 
        metric = "f1_score",
        mode = "max",
        rs: np.random.RandomState=None,
        bound_control = "random") -> None:

        super().__init__(space=space,
            crossover_prob=crossover_prob, 
            mutation_factor=mutation_factor,
            metric=metric,
            mode=mode,
            bound_control=bound_control, 
            rs=rs)
        
        self.min_budget = min_budget
        self.max_budget = max_budget

        self.eta = eta
        
        self._all_in_one = self._get_bracket()
        self._SH_iter = len(self._all_in_one)

        self._genus = None
    
    def _get_bracket(self):
        s_max = int(np.floor(np.log(self.max_budget / self.min_budget) / np.log(self.eta)))

        budgets = (self.max_budget * np.power(self.eta,-np.linspace(start=s_max, stop=0, num=s_max + 1))).tolist()
        budgets = list(map(int, budgets))
        
        N = int(np.ceil(self.eta ** s_max))
        n_configs = [max(int(N*(self.eta**(-i))), 1) for i in range(s_max + 1)]

        bracket = tuple(zip(n_configs, budgets))
        print(bracket)
        return bracket
    
    def _init_eval_genus(self, obj : Callable, **kwargs):
        genus = dict()

        for (pop_size, budget) in self._all_in_one:
            species = dict()
            species["population"] = self._init_population(pop_size)
            species["fitness"] = self._eval_population(obj, species["population"], budget, **kwargs)
            
            #genus[budget] = species
            genus[budget] = self._sort_species(species)
        return genus
    
    def _sort_species(self, species : np.ndarray):
        species = species.copy()
        ranking = np.argsort(species["fitness"])
        if self.mode == "max":
            ranking = ranking[::-1]
        
        species["population"] = species["population"][ranking]
        species["fitness"] = species["fitness"][ranking]

        return species

    def _select_promotions(self, target : dict, previous : dict):
        promotions = []
        pop_size = len(target["population"])

        for individual in previous["population"]:
            # If individual already in target, then ignore it to minimize fn_evals
            if np.any(np.all(individual == target["population"], axis=1)):
                continue
            
            promotions.append(individual)
        
        if len(promotions) >= pop_size:
            promotions = promotions[:pop_size]
        else:
            return previous["population"][:pop_size]
            # raise BufferError("Not enough to promote")
            # Can simply pick top pop_size many individuals even 
            # if duplicates exist in target
        
        return np.asarray(promotions)
    
    def _get_alt_population(self, target : dict, previous : dict):

        # stage == 0, previous is None
        if previous is None:
            
            # Edge case where stage==0, but target population too small
            # for vanilla mutation, so we need an alt_pop that is not None
            if len(target) < self._min_pop_size :
                previous = target
            else:
                return None

        pop_size = len(target["population"])
        alt_pop = previous["population"][:pop_size]

        if len(alt_pop) < self._min_pop_size:
            filler_size = self._min_pop_size - len(alt_pop) + 1
            filler_pop = self._sample(self.global_pupulation, filler_size)

            alt_pop = np.concatenate([filler_pop, alt_pop])
        
        return alt_pop
    
    def optimize(self, obj : Callable, limit : int = 10, unit : str = "iter", **kwargs):
        self._start_timer()

        self.genus = self._init_eval_genus(obj, **kwargs)

        while not self._is_termination(limit, unit): # DEHB iteration
            self._iteration_counter += 1

            for j in range(self._SH_iter): # SH iterations
                
                previous = None
                bracket = self._all_in_one[j:]

                for stage, (pop_size, budget) in enumerate(bracket): # stages in a bracket
                    target =  self.genus[budget]

                    # Only True for first DEHB iteration and non-inital SH stage
                    promotion = True if self._iteration_counter == 0 and stage > 0 else False
                    if promotion:
                        children = self._select_promotions(target, previous)
                    else:
                        alt_pop = self._get_alt_population(target, previous)
                        children = self._next_generation(target["population"], alt_pop)

                    children_fitness = self._eval_population(obj, children, budget, **kwargs)
                    target["population"], target["fitness"] = self._selection(target["population"], children, target["fitness"], children_fitness)

                    target = self._sort_species(target)
                    self.genus[budget] = target
                    previous = target
        
        return self.inc_config
            
    def _init_params(self):
        params = {
            "min_budget" : self.min_budget,
            "max_budget" : self.max_budget,
            "eta"  : self.eta
        }
        params.update(super()._init_params())
        print(params)
        return params

    @property
    def global_pupulation(self):
        assert self.genus is not None, AssertionError("No populaiton in genus initialized")
        return np.concatenate([species["population"] for species in self.genus.values()])




if __name__ == "__main__":

    rs = np.random.RandomState(seed=SEED)
    
    space = ConfigVectorSpace(
        name="neuralnetwork",
        seed=SEED,
        # TODO : find distribution for drop_0, drop_1 and reg_const
        # SUGGESTIONS : drop_0 and drop_1 conditional on layer, as deeper layers need larger dropout
        space={
            "lr": ConfigSpace.UniformFloatHyperparameter("lr", lower=1e-6, upper=1e-1, log=True, default_value=1e-3),
            "dropout_first": ConfigSpace.Float('dropout_first', bounds=(0, 0.9), default=0.34, distribution=ConfigSpace.Normal(mu=0.5, sigma=0.35)),
            "dropout_second": ConfigSpace.Float('dropout_second', bounds=(0, 0.9), default=0.34, distribution=ConfigSpace.Normal(mu=0.5, sigma=0.35)),
            "weight_decay": ConfigSpace.Float("weight_decay", bounds=(0, 5), default=0.1),
            "n_blocks": ConfigSpace.UniformIntegerHyperparameter("n_blocks", lower=2, upper=6, default_value=2),
            "batch_size": ConfigSpace.OrdinalHyperparameter("batch_size", sequence=[64, 128, 512], default_value=64),
            "d_main": ConfigSpace.OrdinalHyperparameter("d_main", sequence=[32, 64, 128,256,512], default_value=128),
            "d_hidden" : ConfigSpace.OrdinalHyperparameter("d_hidden", sequence=[64, 128, 256, 512], default_value=64)
        },
    )

    dehb = DEHB(space, min_budget=50, max_budget=1000, rs=rs)

    start_time = time.process_time()

    print(f"Best configuration  {dehb.optimize(obj, limit=25,  unit='min', dataset_id=40588)}")
    print(f"Time elapsed (CPU time): {(time.process_time() - start_time):.4f} seconds")


((16, 62), (8, 125), (4, 250), (2, 500), (1, 1000))
birds 40588
{'d_in': 260, 'n_blocks': 5, 'd_main': 32, 'd_hidden': 128, 'dropout_first': 0.8377374000639999, 'dropout_second': 0.2342863598269771, 'd_out': 19, 'batch_size': 512, 'epoch': 62}
f1_score: 0.07812442555404611
birds 40588
{'d_in': 260, 'n_blocks': 3, 'd_main': 128, 'd_hidden': 512, 'dropout_first': 0.8568935677327879, 'dropout_second': 0.861572991799429, 'd_out': 19, 'batch_size': 64, 'epoch': 62}
f1_score: 0.11011404673254614
birds 40588
{'d_in': 260, 'n_blocks': 2, 'd_main': 128, 'd_hidden': 512, 'dropout_first': 0.7047907898390531, 'dropout_second': 0.677783945323129, 'd_out': 19, 'batch_size': 512, 'epoch': 62}
f1_score: 0.01754385964912281
birds 40588
{'d_in': 260, 'n_blocks': 3, 'd_main': 64, 'd_hidden': 128, 'dropout_first': 0.3356264456132211, 'dropout_second': 0.8274302466944439, 'd_out': 19, 'batch_size': 512, 'epoch': 62}
f1_score: 0.11330099440605342
birds 40588
{'d_in': 260, 'n_blocks': 3, 'd_main': 128, 'd_hi